In [45]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [46]:
!pip install transformers
import nltk
nltk.download('punkt')
# !pip install nltk


You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [47]:
import torch
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
import random
from nltk import sent_tokenize
from transformers import BertForSequenceClassification

df = pd.read_csv('../input/sentiment-training/Senti_training.csv')
# df.head()
# data=pd.read_excel('issue.xlsx')

In [48]:
df.head()

,Unnamed: 0,Issue Experienced,Sub Reason,Problem statement,Sentiment
0,7,Always very satisfied with Your products. Alth...,other,positive feedback,Positive
1,44,Ease of access and relevant offers that suit m...,other,positive feedback,Positive
2,63,It is easy for me to manage as I am not great ...,other,positive feedback,Positive
3,68,Excellent site,other,positive feedback,Positive
4,145,Easy,other,positive feedback,Positive


In [49]:
data=df[['Issue Experienced', 'Sentiment']]
data['Sentiment'].value_counts()

Positive    4835
Negative    4835
Name: Sentiment, dtype: int64

In [50]:
data['Sentiment']=data['Sentiment'].apply(lambda x: str(x).lower())
#data.loc[data['Sentiment'] == 'covid', 'Sub Reason'] = 'covid-19'
possible_labels = data['Sentiment'].unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
print(label_dict)
data['Sentiment'].value_counts()


{'positive': 0, 'negative': 1}


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


positive    4835
negative    4835
Name: Sentiment, dtype: int64

In [51]:
data['label'] = data['Sentiment'].replace(label_dict)
print(data['Sentiment'].value_counts())
ext=[]
print(data['Sentiment'].value_counts())

positive    4835
negative    4835
Name: Sentiment, dtype: int64
positive    4835
negative    4835
Name: Sentiment, dtype: int64


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [52]:
new_aug_data=[]
for i in data[data['label'].isin(ext)].itertuples():
    
    aug_ext=set(eda(i._1,alpha_rs=0.00001,p_rd=0.00,alpha_ri=0.001))
    
    for j in aug_ext:
        dum=[]
        dum.append(j)
        dum.append(i._2)
        dum.append(i.label)
        new_aug_data.append(dum)
print(data['Sentiment'].value_counts())

positive    4835
negative    4835
Name: Sentiment, dtype: int64


In [53]:
new_df=pd.DataFrame(new_aug_data,columns=['Issue Experienced','Sentiment','label'])
print(new_df['Sentiment'].value_counts())
data=pd.concat([data,new_df])
print(data['Sentiment'].value_counts())

Series([], Name: Sentiment, dtype: int64)
positive    4835
negative    4835
Name: Sentiment, dtype: int64


In [54]:
#data.loc[data['label'].isin([5,10,12,13]), 'label'] = 0

In [55]:
#data=data[data['Sub Reason']!='other']

In [56]:
data

,Issue Experienced,Sentiment,label
0,Always very satisfied with Your products. Alth...,positive,0
1,Ease of access and relevant offers that suit m...,positive,0
2,It is easy for me to manage as I am not great ...,positive,0
3,Excellent site,positive,0
4,Easy,positive,0
...,...,...,...
9665,Told I would be sent a link to reset my passwo...,negative,1
9666,Today I purchases for 90.26 GBP but £5off not ...,negative,1
9667,"Thanks, but I do NOT want an avocado. I’m stil...",negative,1
9668,We are disappointed with the Vegetarian Main c...,negative,1


In [57]:
shuf_data=[]
for i,row in data.iterrows():
  dum=[]
  sent=sent_tokenize(row['Issue Experienced'])
  if len(sent)>1:
    random.shuffle(sent)
    dum.append(''.join(sent))
    dum.append(row['Sentiment'])
    dum.append(row['label'])
    shuf_data.append(dum)

In [58]:
sf=pd.DataFrame(shuf_data,columns=data.columns)

In [59]:
data=pd.concat([data,sf])

In [60]:
print(data['Sentiment'].value_counts())

negative    8234
positive    6244
Name: Sentiment, dtype: int64


In [61]:
data.shape

(14478, 3)

In [62]:
data

,Issue Experienced,Sentiment,label
0,Always very satisfied with Your products. Alth...,positive,0
1,Ease of access and relevant offers that suit m...,positive,0
2,It is easy for me to manage as I am not great ...,positive,0
3,Excellent site,positive,0
4,Easy,positive,0
...,...,...,...
4803,She is 95 but loves clothes.I am trying to rep...,negative,1
4804,Told I would be sent a link to reset my passwo...,negative,1
4805,I’m still not impressed with Sparks offers.Tha...,negative,1
4806,We are disappointed with the Vegetarian Main c...,negative,1


In [63]:
pos=data[data['label']==0]
neg=data[data['label']==1]

In [64]:

print(pos.shape)
print(neg.shape)
neg=neg[0:6244]
print(neg.shape)


(6244, 3)
(8234, 3)
(6244, 3)


In [65]:
frame=[pos,neg]
final=pd.concat(frame)
final['label'].value_counts()

0    6244
1    6244
Name: label, dtype: int64

In [66]:
test = pd.read_csv('../input/sentiment-test-data/Senti_test_data.csv')
test['label']=test['Sentiment'].apply(lambda x: 1 if x =='Negative' else 0)
print(test.columns)
print(final.columns)

Index(['Unnamed: 0', 'Issue Experienced', 'Sub Reason', 'Problem statement',
       'Sentiment', 'label'],
      dtype='object')
Index(['Issue Experienced', 'Sentiment', 'label'], dtype='object')


In [95]:
final['label']=final['label'].apply(lambda x: int(x))
test['label']=test['label'].apply(lambda x: int(x))
final['Issue Experienced']=final['Issue Experienced'].apply(lambda x: str(x))
test['Issue Experienced']=test['Issue Experienced'].apply(lambda x: str(x))

In [96]:
X_train=final['Issue Experienced']
X_val=test['Issue Experienced']
y_train=final['label']
y_val=test['label']

In [97]:
print(X_train)
print(X_val.dtype)
print(y_train.dtype)
print(y_val.dtype)

0       Always very satisfied with Your products. Alth...
1       Ease of access and relevant offers that suit m...
2       It is easy for me to manage as I am not great ...
3                                          Excellent site
4                                                    Easy
                              ...                        
2813    Please advise.The pocket spring single mattres...
2814    Too-way stretch cup T-shirt bras not made any ...
2815    I like the web site but no information about t...
2816    Rosemary ClevettKind regards.I placed an order...
2817    Why are you trying to encourage people in to y...
Name: Issue Experienced, Length: 12488, dtype: object
object
int64
int64


In [98]:
"""from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(data.index.values, 
                                                  data.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=data.label.values)

#data['data_type'] = ['not_set']*data.shape[0]"""

final['data_type'] = 'train'
test['data_type'] = 'val'


In [99]:
#data.groupby(['Sub Reason','label', 'data_type']).count()

In [102]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)
                                          
encoded_data_train = tokenizer.batch_encode_plus(
    final[final.data_type=='train']['Issue Experienced'].values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256,
    truncation=True, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    test[test.data_type=='val']['Issue Experienced'].values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    truncation=True, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(final[final.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(test[test.data_type=='val'].label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [103]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
                                          
# encoded_data_train = tokenizer.batch_encode_plus(
#     data[data.data_type=='train']['filtered SR'].values, 
#     add_special_tokens=True, 
#     return_attention_mask=True, 
#     pad_to_max_length=True, 
#     max_length=500,
#     truncation=True, 
#     return_tensors='pt'
# )

# encoded_data_val = tokenizer.batch_encode_plus(
#     data[data.data_type=='val']['filtered SR'].values, 
#     add_special_tokens=True, 
#     return_attention_mask=True, 
#     pad_to_max_length=True, 
#     max_length=500, 
#     truncation=True, 
#     return_tensors='pt'
# )


# input_ids_train = encoded_data_train['input_ids']
# attention_masks_train = encoded_data_train['attention_mask']
# labels_train = torch.tensor(data[data.data_type=='train'].label.values)

# input_ids_val = encoded_data_val['input_ids']
# attention_masks_val = encoded_data_val['attention_mask']
# labels_val = torch.tensor(data[data.data_type=='val'].label.values)

# dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
# dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [104]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [105]:
# set(eda('i didnot receive the gift', alpha_rs=0.00001,p_rd=0.00,alpha_ri=0.001))
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [106]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [107]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [108]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cpu


In [109]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [110]:
final['Sentiment'].value_counts()

negative    8234
positive    6244
Name: Sentiment, dtype: int64

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    model.train()
    
    loss_train_total = 0
    progress_bar = tqdm(dataloader_train,
                        desc="Epoch {:1d}".format(epoch),
                        leave=False,
                        disable=False)
    
    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        inputs = {
            'input_ids'       : batch[0],
            'attention_mask'  : batch[1],
            'labels'          : batch[2]
        }
    
        outputs = model(**inputs)
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix( {'training_loss': '{:3f}'.format(loss.item() / len(batch))} )

    torch.save(model.state_dict(), 'BERT_ft_epoch{}.model'.format(epoch))
    tqdm.write('\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total / len(dataloader_train)
    tqdm.write('Training loss: {}'.format(loss_train_avg))
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    val_acc=accuracy_per_class(predictions, true_vals)
    
    tqdm.write('Validation loss: {}'.format(val_loss))
    tqdm.write('f1 score (weighted): {}'.format(val_f1))
    # tqdm.write(val_acc)

In [ ]:

# model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
#                                                       num_labels=len(label_dict),
#                                                       output_attentions=False,
#                                                       output_hidden_states=False)
_, predictions, true_vals = evaluate(dataloader_validation)
accuracy_per_class(predictions, true_vals)

In [ ]:
import random
from nltk import sent_tokenize
sen=sent_tokenize('I am b2. hello i am going to kill you. a ')
random.shuffle(sen)
sen

In [ ]:
random.shuffle(['a','b','c'])